In [9]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2
import numpy as np
import time
import mediapipe 
from cvzone.FPS import FPS
import os
import pyautogui as control
control.FAILSAFE =False
import math
import csv
import time

############# our Variables ############
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
imagesList =  []
#get screen shapes
Wsc,Hsc = control.size()
questionNumber = 0
cc = cv2.imread('true.png')

##############load questions #########  
pathCSV = 'mcq.csv' 
with open(pathCSV, newline='\n') as f:
    reader = csv.reader(f)
    dataAll = list(reader)[1:] 
    #print(dataAll) 
totalQuestions = len(dataAll)
###########load marked images #####
path = 'images/'
#print (os.listdir(path))
imagesList =  []
for image in os.listdir(path):
    img = cv2.imread(path + image)
    imagesList.append(img)


######################create mcq class################
class MCQ():
    def __init__(self,data):
        self.question = data[0]
        self.choice1  = data[1]
        self.choice2  = data[2]
        self.choice3  = data[3]
        self.choice4  = data[4]
        self.answer = int(data[5])
        self.userAnswer = None

    def update(self,cursor,boxes):
        #strat get all boundries boxex
        for i ,bbox in enumerate(boxes):
            x1,y1,x2,y2  = bbox 
            if x1<cursor[0]<x2 and y1<cursor[1]<y2:
               self.userAnswer = i+1
               cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),cv2.FILLED)        
#####################start too create object  #############
myList =[]
for question in dataAll :
    myList.append(MCQ(question))

#define  detector
############################### 
detector = HandDetector(staticMode=False,maxHands=2,modelComplexity=1,detectionCon=0.5, minTrackCon=0.5)   

while True :
    _,frame = cap.read()
    frame = cv2.flip(frame, 1)
    #strart show questions and show drwa all fileds 
    if questionNumber<totalQuestions :
            #strat draw all buttoms
            mcq = myList[questionNumber]
            frame ,bbx = cvzone.putTextRect(frame,mcq.question,(50,50),1,1,border=2,offset=20,colorR=(255,0,0))
            frame ,bbx1 = cvzone.putTextRect(frame,mcq.choice1,(50,150),1,1,border=2,offset=20,colorR=(255,0,0))
            frame ,bbx2 = cvzone.putTextRect(frame,mcq.choice2,(300,150),1,1,border=2,offset=20,colorR=(255,0,0))
            frame ,bbx3 = cvzone.putTextRect(frame,mcq.choice3,(50,300),1,1,border=2,offset=20,colorR=(255,0,0))
            frame ,bbx4 = cvzone.putTextRect(frame,mcq.choice4,(300,300),1,1,border=2,offset=20,colorR=(255,0,0))
            # strat detect user answer

            #start detect hand 
            hands,frame= detector.findHands(frame,draw=False,flipType=False)
            if hands :
                hand = hands[0]
                handList = hand['lmList']
                cursor = handList[8][0:2]
                #start detect click action
                distance, info, img = detector.findDistance(handList[8][0:2],handList[12][0:2] ) 
                #print(distance)
                if distance <20 :
                #check user answer
                    mcq.update(cursor,[bbx1,bbx2,bbx3,bbx4])
                    if mcq.userAnswer is not None :
                        time.sleep(.3)
                        if mcq.userAnswer == mcq.answer:
                            frame [200:260,200:260] =imagesList[1]
                        else:
                            frame [200:260,200:260] =imagesList[0]

                        questionNumber += 1
                        time.sleep(.3)
    else:
        #start calculate score
        score = 0 
        for mcq in myList :
            if mcq.userAnswer == mcq.answer :
                frame [200:260,200:260] =imagesList[1]
                score+=1
            
        score =round((score/totalQuestions)*100,2) 
        frame ,_ = cvzone.putTextRect(frame, 'Quiz Completed',(100, 200),1,1, border=2, offset=50,colorR=(255,0,0),)
        frame ,_ = cvzone.putTextRect(frame, f'Your Score{score}%',(350, 200),1,1, border=2, offset=50,colorR=(255,0,0),)
 
    # start Draw Progress Bar
    barvalue =(400//totalQuestions)*questionNumber
    #cv2.rectangle(frame, (0, 380), (400, 400), (255, 0, 0), cv2.FILLED)
    #cv2.rectangle(frame, (0, 380), (400, 400), (0, 255, 0), 2)
    cv2.rectangle(frame, (0, 380), (barvalue, 400), (255, 0, 0), cv2.FILLED)
    cv2.rectangle(frame, (0, 380), (400, 400), (0, 255, 0), 2)
    frame ,_ = cvzone.putTextRect(frame, f'Total Questions {round(questionNumber/totalQuestions,2)*100}%',(410, 395),1,1, border=2, offset=5,colorR=(255,0,0),)
       
    imgPNG = cv2.imread("C:/Users/Geka/Desktop/update.png",cv2.IMREAD_UNCHANGED)
    imgOverlay = cvzone.overlayPNG(frame, imgPNG, pos=[3, 430])
    cv2.imshow('Welcome to Our Test ',frame)

    # Display the camera feed
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows() 
